#Installing Dependencies

In [0]:
!pip install --upgrade onn

In [0]:
!pip install -U imbalanced-learn

##Importing Dependencies

In [0]:
from onn.OnlineNeuralNetwork import ONN
from onn.OnlineNeuralNetwork import ONN_THS
from sklearn.datasets import make_classification, make_circles
from sklearn.model_selection import train_test_split
import torch
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from imblearn.datasets import make_imbalance
import numpy as np

## Initializing Network

In [0]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10)

##Creating Fake Classification Dataset

In [0]:
X, Y = make_classification(n_samples=50000, n_features=10, n_informative=4, n_redundant=0, n_classes=10,
                           n_clusters_per_class=1, class_sep=3)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42, shuffle=True)

##Learning and predicting at the same time

In [85]:
for i in range(len(X_train)):
  onn_network.partial_fit(np.asarray([X_train[i, :]]), np.asarray([y_train[i]]))
  
  if i % 1000 == 0:
    predictions = onn_network.predict(X_test)
    print("Online Accuracy: {}".format(balanced_accuracy_score(y_test, predictions)))

Online Accuracy: 0.11155993574278973
Alpha:[0.83945197 0.04013699 0.04013699 0.04013699 0.04013699]
Training Loss: 1.1259372
Online Accuracy: 0.9660054323032371
Alpha:[0.83991873 0.04002029 0.04002029 0.04002029 0.04002029]
Training Loss: 0.2084848
Online Accuracy: 0.9678860869986063
Alpha:[0.8399446  0.04002719 0.04000937 0.04000937 0.04000937]
Training Loss: 0.17356381
Online Accuracy: 0.9749912036154884
Alpha:[0.83994657 0.04001333 0.04001333 0.04001333 0.04001333]
Training Loss: 0.14811444
Online Accuracy: 0.9760470086129949
Alpha:[0.8399631  0.04000921 0.04000921 0.04000921 0.04000921]
Training Loss: 0.15638338
Online Accuracy: 0.9759461010627343
Alpha:[0.8399948  0.04000129 0.04000129 0.04000129 0.04000129]
Training Loss: 0.15918425
Online Accuracy: 0.9762298100262223
Alpha:[0.83685434 0.04211017 0.04103326 0.04000113 0.04000113]
Training Loss: 0.12820534
Online Accuracy: 0.9734934993336708
Alpha:[0.83354604 0.04303478 0.04209974 0.04084937 0.04047008]
Training Loss: 0.12739542
O

# Learning in batch with CUDA

In [86]:
onn_network = ONN(features_size=10, max_num_hidden_layers=5, qtd_neuron_per_hidden_layer=40, n_classes=10, batch_size=10, use_cuda=True)

Using CUDA :]


In [0]:
from torch.utils.data import Dataset, DataLoader
class Dataset(Dataset):

  def __init__(self, X, Y):
    self.X = X
    self.Y = Y

  def __len__(self):
      return len(self.X)

  def __getitem__(self, idx):
      X = self.X[idx],
      Y = self.Y[idx]

      return X, Y
    
transformed_dataset = Dataset(X_train, y_train)
dataloader = DataLoader(transformed_dataset, batch_size=10,shuffle=True, num_workers=1)

In [88]:
for local_X, local_y in dataloader: 
  onn_network.partial_fit(np.squeeze(torch.stack(local_X).numpy()), local_y.numpy())

Alpha:[0.8398104 0.0400474 0.0400474 0.0400474 0.0400474]
Training Loss: 1.0440425
Alpha:[0.8398949  0.04002627 0.04002627 0.04002627 0.04002627]
Training Loss: 0.22426741
Alpha:[0.8399058  0.04002354 0.04002354 0.04002354 0.04002354]
Training Loss: 0.17775288


In [89]:
predictions = onn_network.predict(X_test)
print("Accuracy: {}".format(balanced_accuracy_score(y_test, predictions)))

Accuracy: 0.9768585011445001


#Using contextual bandit - ONN_THS

In this example the ONN acts like a contextual bandits a reinforcement learning algorithm type. 

In [0]:
X_linear, Y_linear = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)
X_non_linear, Y_non_linear = make_circles(n_samples=10000, noise=0.1, factor=0.3, shuffle=True)
X_linear_2, Y_linear_2 = make_classification(n_samples=10000, n_features=2, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1, class_sep=200, shuffle=True)

X_linear_train = X_linear[:5000]
Y_linear_train = Y_linear[:5000]

X_linear_test = X_linear[5000:]
Y_linear_test = Y_linear[5000:]

X_non_linear_train = X_non_linear[:5000]
Y_non_linear_train = Y_non_linear[:5000]

X_non_linear_test = X_non_linear[5000:]
Y_non_linear_test = Y_non_linear[5000:]

X_linear_train_2 = X_linear_2[:5000]
Y_linear_train_2 = Y_linear_2[:5000]

X_linear_test_2 = X_linear_2[5000:]
Y_linear_test_2 = Y_linear_2[5000:]

In [0]:
gp = ONN_THS(2, 5, 100, 2)

In [92]:
for epoch in range(5):

    for i in range(len(X_linear_train)):
        x = np.asarray([X_linear_train[i, :]])
        y = np.asarray([Y_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_linear_test[i, :]]))[0])
          print("Accuracy: " + str(balanced_accuracy_score(Y_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.06387616 0.06387616 0.21570592 0.32868177 0.32786   ]
Training Loss: 0.06777429
Accuracy: 0.9321726290294914
Alpha:[0.06500413 0.04895455 0.0770593  0.3770126  0.43196943]
Training Loss: 0.115104966
Alpha:[0.06504314 0.04898417 0.0771006  0.37719348 0.43167865]
Training Loss: 0.0
Accuracy: 0.8576111974479568
Alpha:[0.06505971 0.04899694 0.07711567 0.3772635  0.43156415]
Training Loss: 0.0
Alpha:[0.07264031 0.04030425 0.07005572 0.36207    0.4549297 ]
Training Loss: 0.039341945
Accuracy: 0.8541583992380737
Alpha:[0.05263599 0.04004289 0.04004225 0.27240914 0.59486973]
Training Loss: 0.122868255
Alpha:[0.07259597 0.04001465 0.04001465 0.1942651  0.6531096 ]
Training Loss: 1.4758105e-07
Accuracy: 0.8597973189209118
Alpha:[0.08295249 0.04000717 0.0400067  0.14902891 0.68800473]
Training Loss: 0.0
Alpha:[0.08867676 0.04000434 0.04000427 0.12289678 0.70841783]
Training Loss: 0.0
Alpha:[0.0927284  0.0400029  0.04000284 0.1039959  0.72326994]
Training Loss: 0.0
Accuracy: 0.89099187381

In [93]:
for epoch in range(5):

    for i in range(len(X_non_linear_train)):
        x = np.asarray([X_non_linear_train[i, :]])
        y = np.asarray([Y_non_linear_train[i]])

        arm, exp = gp.predict(x)
        
        if arm == y[0]:  
          gp.partial_fit(x, y, exp)
          
        if i % 2000 == 1999:
          pred = []
          print("======================================================")
          for i in range(len(X_linear_test)):  
            pred.append(gp.predict(np.asarray([X_non_linear_test[i, :]]))[0])
          print("Accuracy: " + str(balanced_accuracy_score(Y_non_linear_test, pred)))
          print("======================================================")

print('Finished Training')

Alpha:[0.04125949 0.04869201 0.04964674 0.05104332 0.8093584 ]
Training Loss: 0.047269467
Accuracy: 0.501873123254728
Alpha:[0.04109182 0.04131771 0.0470376  0.0547357  0.8158172 ]
Training Loss: 0.2581089
Accuracy: 0.5027207953741917
Alpha:[0.0407271  0.04077765 0.04092487 0.05368971 0.8238807 ]
Training Loss: 0.25839192
Alpha:[0.04012787 0.04012787 0.04012787 0.04012787 0.8394885 ]
Training Loss: 0.23019107
Accuracy: 0.8586811310427456
Alpha:[0.04000054 0.04000054 0.04000054 0.04000054 0.8399978 ]
Training Loss: 0.058671184
Alpha:[0.04000502 0.04000502 0.04000502 0.04000502 0.83997995]
Training Loss: 0.021879492
Accuracy: 0.9093607175452441
Alpha:[0.04000001 0.04000001 0.04000001 0.04000001 0.84      ]
Training Loss: 0.0035963086
Alpha:[0.04000002 0.04000002 0.04000002 0.04000002 0.8399999 ]
Training Loss: 0.0016941663
Alpha:[0.04       0.04       0.04       0.04       0.84000003]
Training Loss: 0.0010898949
Accuracy: 0.927711338106872
Alpha:[0.04000005 0.04000005 0.04000005 0.040000

# Imbalanced Dataset

In [0]:
X, Y = make_classification(n_samples=10000, n_features=10, n_informative=4, n_redundant=0, n_classes=5, n_clusters_per_class=1, class_sep=200, shuffle=True)

In [0]:
X_t, Y_t = make_imbalance(X, Y, sampling_strategy={0: 800, 1: 200, 2: 900, 3: 500, 4: 1500})

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(X_t, Y_t, test_size=0.2)

In [0]:
gp = ONN_THS(10, 5, 100, 5)

In [98]:
for i in range(len(X_train)):
    x = np.asarray([X_train[i, :]])
    y = np.asarray([y_train[i]])

    arm, exp = gp.predict(x)

    if arm == y[0]:  
      gp.partial_fit(x, y, exp)

    if i % 2000 == 1999:
      pred = []
      print("======================================================")
      for i in range(len(X_test)):  
        pred.append(gp.predict(np.asarray([X_test[i, :]]))[0])
      print("Accuracy: " + str(balanced_accuracy_score(y_test, pred)))
      print("======================================================")

print('Finished Training')

Alpha:[0.04240164 0.0581313  0.20170258 0.37169677 0.32606778]
Training Loss: 0.08427304
Accuracy: 0.9076715772838966
Alpha:[0.04248263 0.05824144 0.20207655 0.37203687 0.32516247]
Training Loss: 2.8180893e-06
Alpha:[0.04252382 0.05829746 0.2022661  0.37219757 0.32471508]
Training Loss: 1.3697098e-06
Accuracy: 0.8584136681505685
Alpha:[0.06496805 0.04386451 0.11642571 0.40504125 0.3697005 ]
Training Loss: 0.05130389
Alpha:[0.06506918 0.04392262 0.11655696 0.40536597 0.36908525]
Training Loss: 7.2467276e-07
Accuracy: 0.9000894639054267
Alpha:[0.06512065 0.04395276 0.11662393 0.40553224 0.36877042]
Training Loss: 4.31895e-07
Alpha:[0.06515684 0.04397394 0.1166664  0.40564492 0.36855793]
Training Loss: 3.182887e-07
Accuracy: 0.9153477176399474
Finished Training


In [0]:
gp = ONN_THS(10, 5, 100, 5)

In [100]:
for i in range(len(X_train_t)):
    x = np.asarray([X_train_t[i, :]])
    y = np.asarray([y_train_t[i]])

    arm, exp = gp.predict(x)

    if arm == y[0]:  
      gp.partial_fit(x, y, exp)

    if i % 2000 == 1999:
      pred = []
      print("======================================================")
      for i in range(len(X_test_t)):  
        pred.append(gp.predict(np.asarray([X_test_t[i, :]]))[0])
      print("Accuracy: " + str(balanced_accuracy_score(y_test_t, pred)))
      print("======================================================")

print('Finished Training')

Alpha:[0.04830293 0.07267473 0.21758947 0.3690484  0.29238445]
Training Loss: 0.100844555
Accuracy: 0.8903208022078465
Alpha:[0.04840219 0.07281803 0.21795925 0.3694929  0.2913276 ]
Training Loss: 1.6427038e-07
Finished Training
